In [6]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import datetime


In [3]:
# load the dataframe with capitals
df = pd.read_csv("concap.csv")

# rename so that the column names are shorter and comply with PEP-8
df.rename(columns={"CountryName": "Country", "CapitalName": "capital", "CapitalLatitude": "lat", "CapitalLongitude": "lon", "CountryCode": "code", "ContinentName": "continent"}, inplace=True)
df.sample(3)


,Country,capital,lat,lon,code,continent
124,Latvia,Riga,56.950000,24.100000,LV,Europe
53,Cocos Islands,West Island,-12.166667,96.833333,CC,Australia
56,Democratic Republic of the Congo,Kinshasa,-4.316667,15.300000,CD,Africa


In [4]:
# to start with let's filter only 2 capitals. Rome and Paris.
ropa = df[df["capital"].isin(["Rome","Paris"])].reset_index()
ropa

,index,Country,capital,lat,lon,code,continent
0,81,France,Paris,48.866667,2.333333,FR,Europe
1,110,Italy,Rome,41.900000,12.483333,IT,Europe


In [2]:
lat_1 = '47.4254570232' # Hochbauamt
lon_1 = '9.37577578175'
lat_2 = '47.4249702' # Liegenschaften
lon_2 = '9.3730005'


In [3]:
# call the OSMR API
r = requests.get(f"http://router.project-osrm.org/route/v1/foot/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")

In [4]:
routes = json.loads(r.content)
route_1 = routes.get("routes")[0]

In [7]:
str(datetime.timedelta(seconds=route_1["duration"]))

'0:01:11.800000'

In [16]:
import datetime

In [5]:
r = requests.get('https://daten.stadt.sg.ch/api/records/1.0/search/?dataset=freie-parkplatze-in-der-stadt-stgallen-pls&q=&geofilter.distance=47.42427%2C9.37186%2C70')

In [17]:
df = json_normalize(json.loads(r.content)['records'])

In [21]:
df[['lat','lon']] = df['geometry.coordinates'].tolist()

In [22]:
df

,datasetid,recordid,record_timestamp,fields.zeitpunkt,fields.belegung_prozent,fields.phid,fields.shortmax,fields.shortfree,fields.shortoccupied,fields.standort,fields.phstate,fields.dist,fields.phname,geometry.type,geometry.coordinates,lat,lon
0,freie-parkplatze-in-der-stadt-stgallen-pls,6e76a2a3d268993aaf5a705f865891d39303d0b3,2021-10-22T07:49:00.368000+00:00,2021-10-22T07:46:37+00:00,88,P22,84,10,74,"[47.424453, 9.371221]",offen,52.2025071449,Rathaus,Point,"[9.371221, 47.424453]",9.371221,47.424453


In [17]:
immo = pd.read_csv('https://raw.githubusercontent.com/cividi/st-gallen-urban-indicators/main/data/price-monitoring/price-monitoring.csv',usecols=['id','rent','lat','lng','street_number'])

In [18]:
immo.head()

,id,rent,street_number,lat,lng
0,108761792,1745,Berneggstrasse 28,47.420898,9.376225
1,2147994348,1720,Heiligkreuzstrasse 53,47.440407,9.393985
2,3000166218,1150,Lindenstrasse 76,47.433056,9.393829
3,3000251780,1390,Feldlistrasse 11,47.421700,9.356135
4,3000402506,2130,Goliathgasse 37,47.427975,9.377123


In [19]:
immo = immo[~immo.lat.isna()]

In [23]:
unisg = ['47.432277', '9.375572']

In [24]:
unisg[1]

'9.375572'

In [49]:
# call the OSMR API
r = json.loads(requests.get(f"http://router.project-osrm.org/route/v1/foot/{immo.iloc[0].lng},{immo.iloc[0].lat};{unisg[1]},{unisg[0]}?overview=false""").content).get("routes")[0]

In [54]:
r['duration']

2022.6

In [62]:
dist = []
dur = []
for index,row in immo.iterrows():
    r = json.loads(requests.get(f"http://router.project-osrm.org/route/v1/foot/{row.lng},{row.lat};{unisg[1]},{unisg[0]}?overview=false""").content).get("routes")[0]
    dist.append(r['distance'])
    dur.append(r['duration'])


In [73]:
immo.insert(loc=len(immo.columns), column='distance_meters',value=dist)

In [75]:
immo.count()

id                  485
rent                485
street_number       485
lat                 485
lng                 485
duration            485
duration_seconds    485
distance_meters     485
dtype: int64

In [71]:
immo.drop(columns='distance_meters',inplace=True)

In [59]:
immo.rename(columns = {'distance':'duration'}, inplace=True)

In [74]:
immo.to_csv('immo_uni.csv')

In [77]:
immo[immo.distance_meters <= 2000]

,id,rent,street_number,lat,lng,duration,duration_seconds,distance_meters
2,3000166218,1150,Lindenstrasse 76,47.433056,9.393829,0:04:09.600000,249.6,1870.4
4,3000402506,2130,Goliathgasse 37,47.427975,9.377123,0:02:56.700000,176.7,1433.4
9,3000836173,1090,Langgasse 89,47.440033,9.391563,0:03:33.400000,213.4,1702.3
11,3000872761,1180,Linsebühlstrasse 16,47.424324,9.380756,0:03:17.200000,197.2,1792.9
12,3000873636,1640,Ludwigstr. 21,47.441250,9.378373,0:03:18.600000,198.6,1647.7
...,...,...,...,...,...,...,...,...
525,3001451526,940,Engelgasse 8,47.426357,9.374825,0:03:59.600000,239.6,1607.2
528,3001454103,1180,Langgasse 46,47.437061,9.388271,0:02:55.800000,175.8,1286.1
529,3001454127,1590,Langgasse 46,47.437061,9.388271,0:02:55.800000,175.8,1286.1
544,3001465599,1240,Moosbruggstrasse 5,47.423676,9.379521,0:03:21.600000,201.6,1823.4


In [88]:
supermarket = pd.read_json('supermarket.json')

In [94]:
supermarket2 = json_normalize(supermarket.features)

In [174]:
sm2 = supermarket2[['id','properties.name','properties.shop','geometry.coordinates']]

In [175]:
ll = pd.DataFrame(sm2['geometry.coordinates'].tolist(), columns=['lat','lon'])

In [176]:
df = pd.concat([sm2,ll],axis=1)

In [178]:
df.count()

id                      35
properties.name         33
properties.shop         35
geometry.coordinates    35
lat                     35
lon                     35
dtype: int64

In [180]:
df.to_csv('../data/supermarkets.csv')

In [122]:
df.count()

properties.name         33
properties.shop         35
geometry.coordinates    35
lat                     35
lon                     35
dtype: int64

In [132]:
immo_test = immo.iloc[:5].copy()

In [155]:
immo.drop(columns=['distance_meters','duration','duration_seconds'],inplace=True)

In [156]:
immo

,id,rent,street_number,lat,lng
0,108761792,1745,Berneggstrasse 28,47.420898,9.376225
1,2147994348,1720,Heiligkreuzstrasse 53,47.440407,9.393985
2,3000166218,1150,Lindenstrasse 76,47.433056,9.393829
3,3000251780,1390,Feldlistrasse 11,47.421700,9.356135
4,3000402506,2130,Goliathgasse 37,47.427975,9.377123
...,...,...,...,...,...
544,3001465599,1240,Moosbruggstrasse 5,47.423676,9.379521
545,3001467270,890,Burgstrasse 80,47.417019,9.356454
546,3001468434,1660,Falkensteinstrasse 72,47.431427,9.398845
547,3001470695,1340,Höhenweg 13,47.429504,9.372371


In [138]:
immo_test.drop(columns=['duration_seconds','duration','distance_meters'], inplace=True)

In [167]:
immo_neu = pd.DataFrame()
for index,row in df.iterrows():
    lat = row.lat
    lng = row.lon
    dist = []
    dur = []
    for index2,row2 in immo.iterrows():
        r = json.loads(requests.get(f"http://router.project-osrm.org/route/v1/foot/{row2.lng},{row2.lat};{lat},{lng}?overview=false""").content).get("routes")[0]
        dist.append(r['distance'])
        dur.append(r['duration'])
        print(r['distance'])
    df_temp = immo.copy()
    df_temp.insert(loc=len(df_temp.columns), column='distance_meters',value=dist)
    df_temp.insert(loc=len(df_temp.columns), column='duration',value=dur)
    df_temp['name'] = row['properties.name']
    df_temp['type'] = row['properties.shop']
    immo_neu = pd.concat([immo_neu,df_temp], ignore_index=True)
    
    

8117.7
9321.7
8228.7
4802.8
7984.3
4957.9
10421.2
4550.8
10376.1
9206.7
10440.1
8343.9
9637.4
5417.9
8361.1
9049.3
8760.5
8959.8
8530.2
9597.6
8903.5
9277.4
4554.2
9155.4
8454.9
9131.7
8361.1
8175.4
3732.7
3755.3
4449.6
10551.2
4673.3
4470.4
4470.4
4470.4
9758.1
7611.8
10051.6
11018.4
9316
9316
8557.5
10376.1
10525.6
9172.7
10595.1
9030.9
11338.8
8684.4
8684.4
4372.8
6764.8
11091.5
4554.2
8469.4
8673.3
2279.9
11153.5
8700
3139.3
7980.6
7980.6
8973.4
4361.7
4740.8
8396.5
8396.5
8971.7
4823.5
10421.2
9262.4
10421.2
9262.4
7755.2
9619.6
8371.8
9931.1
3023.8
7027.3
9216.5
4442.4
7859.4
9222.6
2509.9
8522.4
8594.4
8904.7
8897.9
9321.7
4490.8
7370.5
7870
4602.7
4222.7
8145.7
4422.2
4907.9
3939.9
4361.7
8621.7
4591.2
10248
4409.3
7984.3
8811.6
4265.8
8530.8
7045.6
7994.1
8194.3
9056.1
4614.5
8469.4
9961.2
3214.4
3252.4
8903.7
10621.7
3239.9
8903.5
9022.9
4440.4
8779.4
9360.4
8492.9
2025.1
7686.4
3691.5
9613.9
9587
1835.9
8149.6
4180.3
9307.2
4970.4
8183.6
8549.9
9818.3
4550.8
8522.4
9516.7
85

In [164]:
immo_neu

,id,rent,street_number,lat,lng,distance_meters,duration,name,type
0,108761792,1745,Berneggstrasse 28,47.420898,9.376225,8117.7,593.7,Westcenter,supermarket
1,2147994348,1720,Heiligkreuzstrasse 53,47.440407,9.393985,9321.7,605.4,Westcenter,supermarket
2,3000166218,1150,Lindenstrasse 76,47.433056,9.393829,8228.7,498.0,Westcenter,supermarket
3,3000251780,1390,Feldlistrasse 11,47.421700,9.356135,4802.8,441.6,Westcenter,supermarket
4,3000402506,2130,Goliathgasse 37,47.427975,9.377123,7984.3,565.0,Westcenter,supermarket
...,...,...,...,...,...,...,...,...,...
1450,3001465599,1240,Moosbruggstrasse 5,47.423676,9.379521,20958.8,1085.0,Shopping Arena,supermarket
1451,3001467270,890,Burgstrasse 80,47.417019,9.356454,17646.0,993.1,Shopping Arena,supermarket
1452,3001468434,1660,Falkensteinstrasse 72,47.431427,9.398845,21436.6,1069.6,Shopping Arena,supermarket
1453,3001470695,1340,Höhenweg 13,47.429504,9.372371,20556.1,1054.3,Shopping Arena,supermarket


In [168]:
immo_neu.to_csv('supermarkets_2.csv')

In [ ]:
dist = []
dur = []
for index,row in immo.iterrows():
    r = json.loads(requests.get(f"http://router.project-osrm.org/route/v1/foot/{row.lng},{row.lat};{unisg[1]},{unisg[0]}?overview=false""").content).get("routes")[0]
    dist.append(r['distance'])
    dur.append(r['duration'])

In [ ]:
immo.insert(loc=len(immo.columns), column='distance_meters',value=dist)

In [171]:
immo_neu[immo_neu.distance_meters < 200].loc[,[]]

,id,rent,street_number,lat,lng,distance_meters,duration,name,type
422,3001170136,1510,Kunklerstrasse 1,47.406231,9.304609,91.6,19.9,Westcenter,supermarket
437,3001450273,1730,Kunklerstrasse 1,47.406231,9.304609,91.6,19.9,Westcenter,supermarket
4653,3001396208,1090,Brauerstr. 47,47.437214,9.403699,88.8,11.6,Denner,supermarket
4901,3001100107,1150,Glaserstrasse 3A,47.418877,9.354250,129.2,23.6,Denner,supermarket
4946,3001214688,1530,Waldaustr. 8,47.419987,9.352483,178.6,34.4,Denner,supermarket
...,...,...,...,...,...,...,...,...,...
15057,3000946809,1400,Feldbachstrasse 10,47.420761,9.354719,68.9,10.3,Migros Lachen,supermarket
15086,3001100107,1150,Glaserstrasse 3A,47.418877,9.354250,185.1,33.5,Migros Lachen,supermarket
15089,3001117101,1390,Feldbachstrasse 10,47.420761,9.354719,68.9,10.3,Migros Lachen,supermarket
15380,3001457039,1570,Feldbachstrasse 4,47.421116,9.355559,143.6,21.4,Migros Lachen,supermarket


In [250]:
data = pd.read_csv('../data/ds_kriterien.csv',delimiter=';')

In [238]:
data.Kategorie.unique()

array(['Kindergärten', 'Schulhäuser', 'Aus- und Weiterbildung',
       'Öffentlicher Verkehr', 'Spielgruppen'], dtype=object)

In [252]:
oeff = data[data.Kategorie == 'Schulhäuser'][['Bezeichnung','Geo Point','Kategorie']].copy()

In [253]:
oeff[['lat','lon']] = oeff['Geo Point'].str.split(',').tolist()

In [254]:
oeff.drop(columns='Geo Point', inplace=True)

In [255]:
oeff

,Bezeichnung,Kategorie,lat,lon
8,Schulhaus Boppartshof,Schulhäuser,47.4032063326,9.33406158638
9,Oberstufenschulhaus Engelwies,Schulhäuser,47.4122694611,9.33335450943
10,Schulhaus Riethüsli,Schulhäuser,47.4112171875,9.36504916362
11,Oberstufenschulhaus Zil,Schulhäuser,47.4450745611,9.41008755156
20,Musikschule MSG,Schulhäuser,47.4265557145,9.3808466574
38,Schulhaus Buchwald,Schulhäuser,47.4368430733,9.39301400443
39,Schulhaus Engelwies Primarschule,Schulhäuser,47.4120940801,9.33290226622
40,Schulhaus Grossacker,Schulhäuser,47.4295864139,9.39646623984
41,Schulhaus Krontal,Schulhäuser,47.4339394889,9.39768401654
42,Schulhaus St. Leonhard,Schulhäuser,47.4207501078,9.3688596866


In [247]:
oeff.iloc[5:6]

,Bezeichnung,Kategorie,lat,lon
38,Oberstufenschulhaus Blumenau,Schulhäuser,47.4282329655,9.37914573631


In [256]:
immo_neu = pd.DataFrame()
for index,row in oeff.iterrows():
    lat = row.lat
    lng = row.lon
    dist = []
    dur = []
    for index2,row2 in immo.iterrows():
        r = json.loads(requests.get(f"http://router.project-osrm.org/route/v1/foot/{row2.lng},{row2.lat};{lng},{lat}?overview=false""").content).get("routes")[0]
        dist.append(r['distance'])
        dur.append(r['duration'])
        print(row.Bezeichnung)
    df_temp = immo.copy()
    df_temp.insert(loc=len(df_temp.columns), column='distance_meters',value=dist)
    df_temp.insert(loc=len(df_temp.columns), column='duration',value=dur)
    df_temp['name'] = row['Bezeichnung']
    df_temp['type'] = row['Kategorie']
    immo_neu = pd.concat([immo_neu,df_temp], ignore_index=True)

Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus Boppartshof
Schulhaus 

In [257]:
immo_neu.to_csv('immo_school.csv')